In [48]:
import pickle
import sys
from pprint import pprint
import pandas as pd
import numpy as np

In [9]:
from yelpapi import YelpAPI
yelp_api = YelpAPI('bWBkV7ScDUJxdMPZ1ezI6w', 
                   "uaQ4ocgKwAX9kljB1-5-_94j-Ys", 
                   "ugSGMqKyDcns05Cnn64FgreVyrWPxsf4", 
                   "7ooywRMWOqdd1yCvd7z-ZpUtK7A")
yelp_api

In [10]:
latlong = ['40.75, -74.25|40.91, -74',
        '40.75, -74|40.91, -73.97',
        '40.75, -73.97|40.91, -73.9',
        '40.75, -73.9|40.91, -73.25',
        '40.71, -74.25|40.75, -74',
        '40.71, -74|40.75, -73.97',
        '40.71, -73.97|40.75, -73.9',
        '40.71, -73.9|40.75, -73.25',
        '40.61, -74.25|40.71, -74',
        '40.61, -74|40.71, -73.97',
        '40.61, -73.97|40.71, -73.9',
        '40.61, -73.9|40.71, -73.25',
        '40.51, -74.25|40.61, -74',
        '40.51, -74|40.61, -73.97',
        '40.51, -73.97|40.61, -73.9',
        '40.51, -73.9|40.61, -73.25']

In [11]:
search_result =[]

In [12]:
def parseLocation(searchR):
    return_df = pd.DataFrame()
    for dict_r in searchR:
        #try:
        shop_id = dict_r["id"]
        if 'location' in dict_r.keys():
            dict_loc = dict_r.pop("location")
            address = dict_loc.pop("address")
            display = dict_loc.pop("display_address")
            if 'neighborhoods' in dict_loc.keys():
                neighborhoods = dict_loc.pop('neighborhoods')
                dict_loc["neighborhoods"] = str(neighborhoods)
            if 'coordinate' in dict_loc.keys():
                coordinates = dict_loc.pop("coordinate")
                long = coordinates['longitude']
                lat = coordinates['latitude']
                dict_loc['longitude'] = long
                dict_loc['latitude'] = lat
            dict_loc['address'] = str(address)
            dict_loc['display_address'] = str(display)
            
                #print(dict_loc)
            try:
                df = pd.DataFrame(dict_loc,index=[0])
            except:
                print(dict_loc)
            df["id"] = shop_id
            return_df = pd.concat([return_df,df])
        #except ValueError as err:
        #    print("error: {0}".format(err))
        #    print("Unexpected error:", sys.exc_info()[0])
        #    print(dict_r)
        #    continue
    return return_df

In [13]:
import copy
temp_search = copy.deepcopy(search_result)
location_df = parseLocation(temp_search)
print(len(location_df))

0


In [14]:
data = []
location_data = []
page_size = 20
for bound in latlong:
    print("Processing: ", bound)
    page = 0
    search_result = yelp_api.search_query(category_filter='bars', bounds=bound, limit=page_size, offset=page*page_size).get('businesses')
    location_df = parseLocation(search_result)
    n_biz = len(search_result)
    df = pd.DataFrame(search_result)
#df = pd.concat([df,location_df])
    while n_biz > 1 and page < 49:
        page += 1
        print(page)
        search_result = yelp_api.search_query(category_filter='bars',bounds=bound, limit=page_size, offset=page*page_size).get('businesses')
        location_df2 = parseLocation(search_result)
        df2 = pd.DataFrame(search_result)
    #df2 = pd.concat([df2,location_df2])
        if len(df2) == 0:
            break
        df = df.append(df2,ignore_index=True)
        location_df = location_df.append(location_df2,ignore_index=True)
        
        #with open('yelpapi_df1.pkl','wb') as picklefile:
        #    pickle.dump(df,picklefile)
        #with open('yelpapi_df1_loc.pkl','wb') as picklefile:
        #    pickle.dump(location_df,picklefil
    data.append(df)
    location_data.append(location_df)

Processing:  40.75, -74.25|40.91, -74
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Processing:  40.75, -74|40.91, -73.97
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
Processing:  40.75, -73.97|40.91, -73.9
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
Processing:  40.75, -73.9|40.91, -73.25
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Processing:  40.71, -74.25|40.75, -74
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
Processing:  40.71, -74|40.75, -73.97
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
Processing:  40.71, -73.97|40.75, -73.9
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
Processing:  40.71, -73.9|40.75, -73.25
1
2
3
4
5
6
7
8
9
10
11
12
Processing:  40.61, -74.25|40.71, -74
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Processing:  40.61, -74|40.71, -73.97
1
2
3
4
5
6
7
8
9
10
11
12
Processing:  40.61, -73.97|40.71, -73.9
1
2
3
4


In [15]:
with open('yelpapi_bars.pkl','wb') as picklefile:
    pickle.dump(data,picklefile)
    
with open('yelpapi_bars_location.pkl','wb') as picklefile:
    pickle.dump(location_data,picklefile)

In [16]:
with open('yelpapi_bars.pkl','rb') as picklefile:
    data = pickle.load(picklefile)

with open('yelpapi_bars_location.pkl','rb') as picklefile:
    location_data = pickle.load(picklefile)

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


,address,city,country_code,cross_streets,display_address,geo_accuracy,id,latitude,longitude,neighborhoods,postal_code,state_code
0,['229 Centre St'],Nutley,US,NaN,"['229 Centre St', 'Nutley, NJ 07110']",8.0,cowans-public-nutley,40.812528,-74.156144,NaN,07110,NJ
1,['443 Paterson Ave'],East Rutherford,US,NaN,"['443 Paterson Ave', 'East Rutherford, NJ 07073']",8.0,rockafellas-sports-bar-and-grill-east-rutherfo...,40.840141,-74.100845,NaN,07073,NJ
2,['5 Main Ave'],Wallington,US,NaN,"['5 Main Ave', 'Wallington, NJ 07057']",8.0,cedars-wallington,40.854767,-74.119080,NaN,07057,NJ
3,['258 Park Ave'],East Rutherford,US,NaN,"['258 Park Ave', 'East Rutherford, NJ 07073']",9.5,blarney-station-east-rutherford,40.829008,-74.100459,NaN,07073,NJ
4,['780 Fifth St'],Secaucus,US,NaN,"['780 Fifth St', 'Secaucus, NJ 07094']",8.0,oasis-bar-and-grill-secaucus,40.792965,-74.062804,NaN,07094,NJ
5,['37 Ridge Rd'],Lyndhurst,US,NaN,"['37 Ridge Rd', 'Lyndhurst, NJ 07071']",8.0,wee-willies-sports-bar-and-grille-lyndhurst,40.814950,-74.114310,NaN,07071,NJ
6,['203 Bloomfield Ave'],Bloomfield,US,NaN,"['203 Bloomfield Ave', 'Bloomfield, NJ 07003']",8.0,the-bar-bloomfield,40.781810,-74.190570,NaN,07003,NJ
7,['414 Main Ave'],Clifton,US,NaN,"['414 Main Ave', 'Clifton, NJ 07014']",8.0,the-rock-bar-and-grill-clifton,40.839725,-74.134148,NaN,07014,NJ
8,['132 Essex St'],Lodi,US,NaN,"['132 Essex St', 'Lodi, NJ 07644']",8.0,rebar-and-kitchen-lodi-2,40.892185,-74.073669,NaN,07644,NJ
9,['340 Paterson Ave'],East Rutherford,US,NaN,"['340 Paterson Ave', 'East Rutherford, NJ 07073']",8.0,garden-state-ale-house-east-rutherford,40.836845,-74.099540,NaN,07073,NJ


In [17]:
total_df = pd.DataFrame()
for df in data:
    total_df = pd.concat([total_df,df],axis=0)

In [23]:
total_location_df = pd.DataFrame()
for df in location_data:
    total_location_df = pd.concat([total_location_df,df],axis=0)

In [30]:
type(total_location_df)

pandas.core.frame.DataFrame

In [32]:
df_total = total_df.merge(total_location_df,how='left',on='id')

In [33]:
df_total.shape

(5092, 32)

In [40]:
df_total_drop = df_total.drop(['deals', 
                    'gift_certificates',
                    'image_url',
                    'menu_date_updated',
                    'menu_provider', 
                    'mobile_url', 
                    'rating_img_url', 
                    'rating_img_url_large', 
                    'rating_img_url_small',
                    'snippet_image_url',
                    'snippet_text', 
                    'address',
                    'city', 
                    'country_code',
                    'cross_streets',
                    'display_address',
                    'geo_accuracy', 
                    'neighborhoods', 
                    'postal_code',
                    'state_code'],axis=1)

In [41]:
df_total_drop.head()

,categories,display_phone,id,is_claimed,is_closed,name,phone,rating,review_count,url,latitude,longitude
0,"[[Pubs, pubs], [American (New), newamerican]]",+1-973-542-8151,cowans-public-nutley,True,False,Cowan's Public,9735428151,4.5,194,https://www.yelp.com/biz/cowans-public-nutley?...,40.812528,-74.156144
1,"[[Sports Bars, sportsbars]]",+1-201-842-0777,rockafellas-sports-bar-and-grill-east-rutherfo...,True,False,Rockafella's Sports Bar & Grill,2018420777,4.0,21,https://www.yelp.com/biz/rockafellas-sports-ba...,40.840141,-74.100845
2,"[[Hookah Bars, hookah_bars], [Lebanese, lebane...",+1-973-470-8880,cedars-wallington,True,False,Cedars,9734708880,4.5,84,https://www.yelp.com/biz/cedars-wallington?adj...,40.854767,-74.119080
3,"[[Pubs, pubs], [Irish, irish]]",+1-201-531-0001,blarney-station-east-rutherford,False,False,Blarney Station,2015310001,4.0,70,https://www.yelp.com/biz/blarney-station-east-...,40.829008,-74.100459
4,"[[Bars, bars], [American (Traditional), tradam...",+1-201-863-6900,oasis-bar-and-grill-secaucus,True,False,Oasis Bar & Grill,2018636900,4.5,27,https://www.yelp.com/biz/oasis-bar-and-grill-s...,40.792965,-74.062804


In [49]:
from scipy import spatial

In [50]:
df_ct = pd.read_csv('geoid_census_tract.csv')
df_ct['COORDS'] = list(zip(df_ct.LAT, df_ct.LON))
df_ct.COORDS = df_ct.COORDS.apply(list)
df_ct.head()

,INDEX,GEOID,STATE,COUNTY,TRACT,LAT,LON,COORDS
0,88,36005000100,36,5,100,40.793492,-73.883532,"[40.7934921, -73.8835318]"
1,89,36005000200,36,5,200,40.804573,-73.856859,"[40.8045733, -73.8568585]"
2,90,36005000400,36,5,400,40.808915,-73.850488,"[40.8089152, -73.8504884]"
3,91,36005001600,36,5,1600,40.818848,-73.858076,"[40.8188478, -73.8580764]"
4,92,36005001900,36,5,1900,40.800999,-73.909373,"[40.800999, -73.9093729]"


In [51]:
X_train = pd.Series(df_ct.COORDS).tolist()
y_train = df_ct.GEOID

def nearest(X_train, y_train, X_test):
    tree = spatial.KDTree(np.array(X_train))
    (distances,indexes) = tree.query(X_test)
    for i,d in list(zip(indexes,distances)):
        yield y_train[i]

In [53]:
df_total_drop['coordinates'] = list(zip(df_total_drop.latitude, df_total_drop.longitude))
df_total_drop.coordinates = df_total_drop.coordinates.apply(list)

print('pass1')

X_test = [df_total_drop.coordinates]
results = []
for result in nearest(X_train, y_train, X_test):
    results.append(result)

print(len(results)) 

df_coords = pd.DataFrame(np.asarray(results))
df_coords = df_coords.transpose()

print(df_coords.shape)

pass1
1
(5092, 1)


In [54]:
df_total_drop['census_tract'] = df_coords

In [55]:
df_total_drop

,categories,display_phone,id,is_claimed,is_closed,name,phone,rating,review_count,url,latitude,longitude,coordinates,census_tract
0,"[[Pubs, pubs], [American (New), newamerican]]",+1-973-542-8151,cowans-public-nutley,True,False,Cowan's Public,9735428151,4.5,194,https://www.yelp.com/biz/cowans-public-nutley?...,40.812528,-74.156144,"[40.8125279099, -74.1561444104]",3.606101e+10
1,"[[Sports Bars, sportsbars]]",+1-201-842-0777,rockafellas-sports-bar-and-grill-east-rutherfo...,True,False,Rockafella's Sports Bar & Grill,2018420777,4.0,21,https://www.yelp.com/biz/rockafellas-sports-ba...,40.840141,-74.100845,"[40.8401412964, -74.1008453369]",3.606101e+10
2,"[[Hookah Bars, hookah_bars], [Lebanese, lebane...",+1-973-470-8880,cedars-wallington,True,False,Cedars,9734708880,4.5,84,https://www.yelp.com/biz/cedars-wallington?adj...,40.854767,-74.119080,"[40.8547668457, -74.1190795898]",3.606101e+10
3,"[[Pubs, pubs], [Irish, irish]]",+1-201-531-0001,blarney-station-east-rutherford,False,False,Blarney Station,2015310001,4.0,70,https://www.yelp.com/biz/blarney-station-east-...,40.829008,-74.100459,"[40.829008406, -74.1004594467]",3.606101e+10
4,"[[Bars, bars], [American (Traditional), tradam...",+1-201-863-6900,oasis-bar-and-grill-secaucus,True,False,Oasis Bar & Grill,2018636900,4.5,27,https://www.yelp.com/biz/oasis-bar-and-grill-s...,40.792965,-74.062804,"[40.7929645479, -74.0628042072]",3.606101e+10
5,"[[Sports Bars, sportsbars], [American (Traditi...",+1-201-939-7006,wee-willies-sports-bar-and-grille-lyndhurst,True,False,Wee Willie's Sports Bar & Grille,2019397006,4.5,11,https://www.yelp.com/biz/wee-willies-sports-ba...,40.814950,-74.114310,"[40.81495, -74.11431]",3.606101e+10
6,"[[Sports Bars, sportsbars], [Lounges, lounges]]",+1-973-707-5005,the-bar-bloomfield,True,False,The Bar,9737075005,4.5,19,https://www.yelp.com/biz/the-bar-bloomfield?ad...,40.781810,-74.190570,"[40.78181, -74.1905699]",3.608503e+10
7,"[[Sports Bars, sportsbars]]",+1-973-916-1888,the-rock-bar-and-grill-clifton,False,False,The Rock Bar & Grill,9739161888,4.0,30,https://www.yelp.com/biz/the-rock-bar-and-gril...,40.839725,-74.134148,"[40.8397254944, -74.134147644]",3.606101e+10
8,"[[Sports Bars, sportsbars], [Burgers, burgers]...",+1-201-368-8181,rebar-and-kitchen-lodi-2,True,False,ReBAR & Kitchen,2013688181,4.0,214,https://www.yelp.com/biz/rebar-and-kitchen-lod...,40.892185,-74.073669,"[40.8921852112, -74.0736694336]",3.606102e+10
9,"[[Sports Bars, sportsbars], [American (New), n...",+1-201-528-7767,garden-state-ale-house-east-rutherford,True,False,Garden State Ale House,2015287767,3.5,72,https://www.yelp.com/biz/garden-state-ale-hous...,40.836845,-74.099540,"[40.836845, -74.09954]",3.606101e+10


In [56]:
with open('yelpapi_bars_wGEOID.pkl','wb') as picklefile:
    pickle.dump(df_total_drop,picklefile)
    
with open('yelpapi_bars_wGEOID.pkl','rb') as picklefile:
    df = pickle.load(picklefile)

In [ ]:
# for sharing

In [58]:
df['count'] = 1

In [60]:
df_ = df.drop(['categories', 'display_phone', 'id', 'is_claimed', 'is_closed', 'name',
       'phone', 'rating', 'review_count', 'url', 'latitude', 'longitude',
       'coordinates'],axis=1)

In [64]:
df_g = df_.groupby('census_tract',as_index=False).sum()

In [65]:
df_g.to_csv('yelp_bars.csv')

In [39]:
# biz_names = total_df.id
# l = list(set(biz_names.values))
# len(l)

4936

In [40]:
# dict_biz = [yelp_api.business_query(id=i) for i in l]
# len(dict_biz)

4936

In [41]:
# df_biz_bar = pd.DataFrame(dict_biz)

# with open('yelpapi_biz_bar.pkl','wb') as picklefile:
#     pickle.dump(df_biz_bar,picklefile)

# len(df_biz_bar)

4936

In [42]:
# with open("yelpapi_biz_bar.pkl", 'rb') as picklefile: 
#     biz_bar = pickle.load(picklefile)
# print(len(biz_bar))
# biz_bar.columns

4936


Index(['categories', 'deals', 'display_phone', 'gift_certificates', 'id',
       'image_url', 'is_claimed', 'is_closed', 'location', 'menu_date_updated',
       'menu_provider', 'mobile_url', 'name', 'phone', 'rating',
       'rating_img_url', 'rating_img_url_large', 'rating_img_url_small',
       'review_count', 'reviews', 'snippet_image_url', 'snippet_text', 'url'],
      dtype='object')

In [43]:
# l_list = []
# l_series = biz_bar.reviews
# for i in range(0,len(l_series)):
#     l_list.append(l_series[i][0]['time_created'])
# print(len(np.asarray(l_list)))
# print(len(biz_bar))

NameError: name 'np' is not defined

In [ ]:
# biz_bar['review_date'] = l_list
# biz_bar.head()

In [ ]:
# biz_bar = df.drop(['deals',
#                   'gift_certificates', 
#                   'image_url',  
#                   'menu_date_updated',
#                   'menu_provider', 
#                   'mobile_url',
#                   'rating_img_url', 
#                   'rating_img_url_large',
#                   'rating_img_url_small',
#                   'snippet_image_url',
#                   'snippet_text'],axis=1)